# **Advanced variant filtering**

In [2]:
from pyoskar.core import Oskar
from pyoskar.sql import *
from pyoskar.analysis import *
from pyspark.sql.functions import col, udf, count, explode, concat, when, expr
from pyspark.sql.functions import *

oskar = Oskar(spark)
df = oskar.load("/home/roldanx/appl/oskar/oskar-spark/src/test/resources/platinum_chr22.small.parquet")

TypeError: 'JavaPackage' object is not callable

## Hardy Weinberg

In [2]:
oskar.hardyWeinberg(df,"hgvauser@platinum:illumina_platinum").select("id", "HWE").show(10)

NameError: name 'oskar' is not defined

## Inbreeding coefficient

In [ ]:
df2 = oskar.stats(df, studyId="hgvauser@platinum:illumina_platinum", missingAsReference=True)
oskar.inbreedingCoefficient(df2).show(10)

## Mendelian error

In [ ]:
oskar.mendel(df, "NA12877", "NA12878", "NA12879").select("id", "mendelianError").filter(col("mendelianError") != "0").show()